In [1]:
import os
from langchain import HuggingFaceHub, PromptTemplate
hf_key = os.environ.get('huggingface_token', None)

In [2]:
llm = HuggingFaceHub(repo_id = 'Salesforce/codegen-350M-mono', 
                     model_kwargs = {
                         "temperature" : 0,
                         "max_length" : 1024,
                     })

/workspaces/RAG_secure_code_generation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspaces/RAG_secure_code_generation/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
template = '''def sum_two_numbers(num1 : int, num2 : int)->int:
    """Given two numbers, return the sum of them."""
'''